<a href="https://colab.research.google.com/github/bhoo-git/bhoo-git/blob/main/IDL_BCI_EEG_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IDL Project: Predicting BCI_EEG signals using Deep Learning**

In [1]:
!pip install wandb torchsummaryX --quiet
import torch
import torch.nn as nn
from torchsummaryX import summary
import torchvision
import os
import gc
from tqdm import tqdm
from PIL import Image
import numpy as np
import pandas as pd
import glob
import wandb
import matplotlib.pyplot as plt
import multiprocessing as mp
import scipy
import seaborn as sns
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Num_Workers: {mp.cpu_count()}')
print("Device: ", DEVICE)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 21.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.4 MB/s eta 0:00:00
Num_Workers: 2
Device:  cuda


In [2]:
# Get BCI_EEG Data 
!mkdir /content/bci_data
!wget -q https://cmu.box.com/shared/static/dje4whisfwszhe2vvfphvahzr563kp2x.zip --content-disposition --show-progress
!unzip -qo 'BCI_EEG_data.zip' -d '/content/bci_data'

"""
Original Data From:
 *https://figshare.com/articles/online_resource/Shared_data_for_exploring_training_effect_in_42_human_subjects_using_a_noninvasive_sensorimotor_rhythm-based_online_BCI/7959572*
"""

BCI_EEG_data.zip    100%[===================>]   4.84G  13.1MB/s    in 7m 14s  


'\nOriginal Data From:\n *https://figshare.com/articles/online_resource/Shared_data_for_exploring_training_effect_in_42_human_subjects_using_a_noninvasive_sensorimotor_rhythm-based_online_BCI/7959572*\n'

# **EDA**

---
...Each file includes the online results from the BCI experimentation for each run (saved in a cell variable ‘BCI_UseResults’), key parameters for the experiment (saved in a structure ‘Experiment_Parm’), key parameters for the state of the raw EEG signal (saved in a structure ‘Experimental_states’), and the raw EEG signal (saved in a variable ‘output_data’).

The raw EEG signals for experiments one and two are composed of 62 channels of EEG data with a sampling frequency of 100Hz, while data for experiment three contains 64 channels of EEG sampled at 128Hz...

In [2]:
config = {
    'epochs'        : 20,
    'batch_size'    : 16,
    'lr'            : 5e-4, # 0.1 for SGD, 5e-4 for adamW
    'patience'      : 2,
    'lr_decay'      : 0.1,
    'weight_decay'  : 0.001,
    'momentum'      : 0.9,
    'nesterov'      : True,
    'drop_rate'     : 0.25,
    'label_smooth'  : 0.1,
    'drop_path'     : 0.1
}

In [3]:
class SubjectDataset(torch.utils.data.Dataset):

    def __init__(self, root, subject = 1, mode = 'train'):
        
        self.eegs, self.results = [], []
        for experiment in ["Exp1","Exp2"]:
            dir = root + '/' + experiment
            for session in os.listdir(dir):
                should_load = f'Subj{subject}_' in session
                if mode == 'train':
                    should_load = not should_load
                if should_load:
                    # Load a single mat file
                    mat = scipy.io.loadmat(dir+'/'+session)
                    # Get raw EEG data in one session
                    session_eeg = mat['output_data']   
                    # Get target code in one session
                    exp_states = dict()
                    for _ in mat['Experimental_states']:
                        for wrapper in _:
                            fieldnames = wrapper.dtype.names
                            for idx, val in enumerate(wrapper):
                                exp_states[f'{fieldnames[idx]}'] = val.flatten()
                    session_targetCode = exp_states['TargetCode']-1
                    '''
                    TODO: The different effective length of 3 sessions will cause problems for data loader.
                    --> for now, we use a fix number 100 as the effective duration
                    --> If this impacts the model performance, we can try pad_pack_sequence
                    '''
                    
                    # Remove useless data
                    trial_start_index = np.nonzero(np.diff(session_targetCode))[0][::2]+1
                    trial_start_end_index = np.nonzero(np.diff(session_targetCode))[0]+1

                    # Split continuous eeg data to trials
                    eeg_split_trials = np.split(session_eeg,trial_start_end_index)[1::2]
                    
                    # Motor imgination starts 2s after the target shows up. Plus 0.1s for reactions time
                    effective_start = 210
                    
                    # The target freezes for 1s after hitting the target, signals in this 1s is useless.
                    # Use the min duration time - 1s as the effective end time. 
                    # Different sessions have different min duration time
                    # min_trial_duration = np.amin(np.diff(trial_start_end_index)[::2])
                    # effective_end = min_trial_duration - 100
                    
                    # Update: for now, we use a fix number 100 as the effective duration
                    effective_end = effective_start + 100
                    
                    effective_eeg = []
                    for eeg in eeg_split_trials:
                        effective_eeg.append(eeg[effective_start:effective_end,:])
                    
                    # Remove useless target code
                    session_targetCode = session_targetCode[trial_start_index]
                    
                    # Add the session data into the array
                    self.eegs += (effective_eeg)
                    self.results.append(session_targetCode)
        
        # Concatenate results
        self.results = np.concatenate(self.results)    
        self.length = len(self.results)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        eegs = np.array(self.eegs[ind]) # extra [] to make an additional dimension for channel #REMOVED
        eegs = torch.FloatTensor(eegs)
        results = torch.tensor(self.results[ind])
        return eegs, results

    '''
    TODO: Discuss if we need batch-wise operations.
    '''
    # def collate_fn(batch):
    #     return batch

In [4]:
# Test of dataset and data loader
root = '/content/bci_data'
train_dataset = SubjectDataset(root, subject=2, mode = 'train')
train_loader = torch.utils.data.DataLoader(
    dataset     = train_dataset, 
    num_workers = 2,
    batch_size  = 16, 
    pin_memory  = True,
    shuffle     = True,
    # collate_fn  = AudioDataset.collate_fn
)

test_dataset = SubjectDataset(root, subject=2, mode = 'test')
test_loader = torch.utils.data.DataLoader(
    dataset     = test_dataset, 
    num_workers = 2,
    batch_size  = 16, 
    pin_memory  = True,
    shuffle     = False,
    # collate_fn  = AudioDataset.collate_fn
)

In [5]:
print("No. of data         : ", train_dataset.__len__())
print("No. of test data    : ", test_dataset.__len__())
print("Shape of data       : ", train_dataset[0][0].shape)

print("Train batches       : ", train_loader.__len__())
print("Test batches        : ", test_loader.__len__())

# Test of dataset and data loader
for x, y in train_loader:
    print(x.shape, y.shape)
    xt, yt = x, y
    break
print(xt.shape)

No. of data         :  9945
No. of test data    :  375
Shape of data       :  torch.Size([100, 62])
Train batches       :  622
Test batches        :  24
torch.Size([16, 100, 62]) torch.Size([16])
torch.Size([16, 100, 62])


# Model:CustomNet (Conv1d embedder - Transformer Encoder - Classification Head)

In [6]:
# Utils
class PermuteBlock(torch.nn.Module):
    def forward(self, x):
        return x.transpose(1, 2)


class LockedDropout(torch.nn.Module):
    """ LockedDropout applies the same dropout mask to every time step.

    **Thank you** to Sales Force for their initial implementation of :class:`WeightDrop`. Here is
    their `License
    <https://github.com/salesforce/awd-lstm-lm/blob/master/LICENSE>`__.

    Args:
        p (float): Probability of an element in the dropout mask to be zeroed.
    """

    def __init__(self, p=0.4):
        self.p = p
        super().__init__()

    def forward(self, x, batch_size=config['batch_size']):
        """
        Args:
            x (:class:`torch.FloatTensor` [sequence length, batch size, rnn hidden size]): Input to
                apply dropout too.
        """
        if not self.training or not self.p:
            return x
        x = x.clone()
        mask = x.new_empty(batch_size, x.size(1), requires_grad=False).bernoulli_(1 - self.p)
        mask = mask.div_(1 - self.p)
        mask = mask.expand_as(x)
        return x * mask


    def __repr__(self):
        return self.__class__.__name__ + '(' \
            + 'p=' + str(self.p) + ')'


class pBLSTM(torch.nn.Module):
    '''
    Pyramidal BiLSTM
    At each step,
    1. Pad your input if it is packed (Unpack it)
    2. Reduce the input length dimension by concatenating feature dimension
        (Tip: Write down the shapes and understand)
        (i) How should  you deal with odd/even length input? 
        (ii) How should you deal with input length array (x_lens) after truncating the input?
    3. Pack your input
    4. Pass it into LSTM layer
    '''
    def __init__(self, input_size, hidden_size, dropout):
        super(pBLSTM, self).__init__()
        self.blstm = torch.nn.LSTM(2 * input_size, hidden_size, bidirectional=True, batch_first=True, dropout=dropout, device=DEVICE) 
        # TODO: Initialize a single layer bidirectional LSTM with the given input_size and hidden_size

    def forward(self, x_packed): # x_packed is a PackedSequence
        x_unpack, x_lens = pad_packed_sequence(x_packed, batch_first=True)          # TODO: Pad Packed Sequence
        x_unpack, x_lens = self.trunc_reshape(x_unpack, x_lens)                                   # Call self.trunc_reshape()
        x_pack = pack_padded_sequence(x_unpack, x_lens, batch_first=True, enforce_sorted=False)           # TODO: Pack Padded Sequence. What output(s) would you get?
        out, (h_n, c_n) = self.blstm(x_pack)                                                                         # TODO: Pass the sequence through bLSTM
        return out

    def trunc_reshape(self, x, x_lens): 
        B, T, C = x.shape                          # TODO: If you have odd number of timesteps, how can you handle it? (Hint: You can exclude them)
        if T % 2 == 1:  x = x[:,:-1,:]
        x = x.reshape(B, T // 2, C *2)    # TODO: Reshape x. When reshaping x, you have to reduce number of timesteps by a downsampling factor while increasing number of features by the same factor
        x_lens = x_lens // 2                    # TODO: Reduce lengths by the same downsampling factor
        return x, x_lens  

In [37]:
class CustomNet(nn.Module):
    def __init__(self, in_channels=62, hid_dim=6, n_classes=2, n_layers=5,dropout=0.5):
        super().__init__()
        self.embedder = nn.Sequential(#PermuteBlock(),
            nn.Linear(in_channels, in_channels*2),
            nn.LayerNorm(in_channels*2), torch.nn.GELU(),
            nn.Linear(in_channels*2, in_channels*4),
            nn.LayerNorm(in_channels*4), torch.nn.GELU(),
           )#PermuteBlock())

        self.rnn = nn.LSTM(in_channels*4, hid_dim, num_layers=n_layers, batch_first=True,
                           dropout=dropout)
        self.class_head = nn.Sequential(
            nn.Linear(hid_dim, n_classes),
        )
        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight, gain=1)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight, gain=1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.embedder(x)
        x, _ = self.rnn(x)
        x = self.class_head(x)
        return x[:,-1,:]

In [46]:
model = CustomNet(in_channels=62, hid_dim=4, n_classes=2,n_layers=6, dropout=0.4).to(DEVICE)
print(model)
xt = xt.cuda()
summary(model, xt)

CustomNet(
  (embedder): Sequential(
    (0): Linear(in_features=62, out_features=124, bias=True)
    (1): LayerNorm((124,), eps=1e-05, elementwise_affine=True)
    (2): GELU(approximate='none')
    (3): Linear(in_features=124, out_features=248, bias=True)
    (4): LayerNorm((248,), eps=1e-05, elementwise_affine=True)
    (5): GELU(approximate='none')
  )
  (rnn): LSTM(248, 4, num_layers=6, batch_first=True, dropout=0.4)
  (class_head): Sequential(
    (0): Linear(in_features=4, out_features=2, bias=True)
  )
)
                       Kernel Shape    Output Shape  Params Mult-Adds
Layer                                                                
0_embedder.Linear_0       [62, 124]  [16, 100, 124]  7.812k    7.688k
1_embedder.LayerNorm_1        [124]  [16, 100, 124]   248.0     124.0
2_embedder.GELU_2                 -  [16, 100, 124]       -         -
3_embedder.Linear_3      [124, 248]  [16, 100, 248]   31.0k   30.752k
4_embedder.LayerNorm_4        [248]  [16, 100, 248]   496.0    

/usr/local/lib/python3.9/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_embedder.Linear_0,"[62, 124]","[16, 100, 124]",7812.0,7688.0
1_embedder.LayerNorm_1,[124],"[16, 100, 124]",248.0,124.0
2_embedder.GELU_2,-,"[16, 100, 124]",NaN,NaN
3_embedder.Linear_3,"[124, 248]","[16, 100, 248]",31000.0,30752.0
4_embedder.LayerNorm_4,[248],"[16, 100, 248]",496.0,248.0
5_embedder.GELU_5,-,"[16, 100, 248]",NaN,NaN
6_rnn,-,"[16, 100, 4]",4864.0,4672.0
7_class_head.Linear_0,"[4, 2]","[16, 100, 2]",10.0,8.0


In [33]:
criterion = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=config['lr'], momentum=config['momentum'], weight_decay=config['weight_decay'])
optimizer = torch.optim.AdamW(model.parameters(), lr=config['lr']) #, weight_decay=0.001)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(patience = 5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
         optimizer=optimizer,
         T_max = config['epochs'],
         verbose=True
 )
scaler = torch.cuda.amp.GradScaler()

Adjusting learning rate of group 0 to 5.0000e-04.


In [19]:
def train(model, dataloader, optimizer, criterion):
    
    model.train()

    # Progress Bar 
    batch_bar   = tqdm(total=len(dataloader), dynamic_ncols=True, leave=False, position=0, desc='Train', ncols=5) 

    num_correct = 0
    total_loss  = 0

    for i, (x, y) in enumerate(dataloader):
        
        optimizer.zero_grad() # Zero gradients

        x, y = x.to(DEVICE), y.to(DEVICE)
        
        with torch.cuda.amp.autocast(): # This implements mixed precision. Thats it! 
            outputs = model(x)
            loss    = criterion(outputs, y)

        # Update no. of correct predictions & loss as we iterate
        num_correct     += int((torch.argmax(outputs, axis=1) == y).sum())
        total_loss      += float(loss.item())

        # tqdm lets you add some details so you can monitor training as you train.
        batch_bar.set_postfix(
            acc         = "{:.04f}%".format(100 * num_correct / (config['batch_size']*(i + 1))),
            loss        = "{:.04f}".format(float(total_loss / (i + 1))),
            num_correct = num_correct,
            lr          = "{:.04f}".format(float(optimizer.param_groups[0]['lr']))
        )
        
        loss.backward()
        optimizer.step()
        # scaler.scale(loss).backward() # This is a replacement for loss.backward()
        # scaler.step(optimizer) # This is a replacement for optimizer.step()
        # scaler.update() 

        # TODO? Depending on your choice of scheduler,
        # You may want to call some schdulers inside the train function. What are these?
      
        batch_bar.update() # Update tqdm bar

    batch_bar.close() # You need this to close the tqdm bar

    acc         = 100 * num_correct / (config['batch_size']* len(dataloader))
    total_loss  = float(total_loss / len(dataloader))

    return acc, total_loss

In [20]:
def validate(model, dataloader, criterion):
  
    model.eval()
    batch_bar = tqdm(total=len(dataloader), dynamic_ncols=True, position=0, leave=False, desc='Val', ncols=5)

    num_correct = 0.0
    total_loss = 0.0

    for i, (x, y) in enumerate(dataloader):
        
        # Move images to device
        x, y = x.to(DEVICE), y.to(DEVICE)
        
        # Get model outputs
        with torch.inference_mode():
            outputs = model(x)
            loss = criterion(outputs, y)

        num_correct += int((torch.argmax(outputs, axis=1) == y).sum())
        total_loss += float(loss.item())

        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * num_correct / (config['batch_size']*(i + 1))),
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            num_correct=num_correct)

        batch_bar.update()
        
    batch_bar.close()
    acc = 100 * num_correct / (config['batch_size']* len(dataloader))
    total_loss = float(total_loss / len(dataloader))
    return acc, total_loss

In [ ]:
best_valacc = 0.0

for epoch in range(config['epochs']):

    curr_lr = float(optimizer.param_groups[0]['lr'])

    # train_acc, train_loss = train(model, train_loader, optimizer, criterion)
    train_acc, train_loss = train(model, train_loader, optimizer, criterion)
    
    print("\nEpoch {}/{}: \nTrain Acc {:.04f}%\t Train Loss {:.04f}\t Learning Rate {:.04f}".format(
        epoch + 1,
        config['epochs'],
        train_acc,
        train_loss,
        curr_lr))
    
    # val_acc, val_loss = validate(model, valid_loader, criterion)
    val_acc, val_loss = validate(model, test_loader, criterion)
    
    print("Val Acc {:.04f}%\t Val Loss {:.04f}".format(val_acc, val_loss))

    # wandb.log({"train_loss":train_loss, 'train_Acc': train_acc, 'validation_Acc':val_acc, 
    #            'validation_loss': val_loss, "learning_Rate": curr_lr})
    
    # If you are using a scheduler in your train function within your iteration loop, you may want to log
    # your learning rate differently
    # scheduler.step(val_loss)
    scheduler.step()

    #Save model in drive location if val_acc is better than best recorded val_acc
    # if val_acc >= best_valacc:
    #   #path = os.path.join(root, model_directory, 'checkpoint' + '.pth')
    #   print("Saving model")
    #   torch.save({'model_state_dict':arcfacemodel.model.state_dict(),
    #               'optimizer_state_dict':optimizer.state_dict(),
    #               'scheduler_state_dict':scheduler.state_dict(),
    #               'val_acc': val_acc, 
    #               'epoch': epoch}, './checkpoint.pth')
    #   best_valacc = val_acc
    #   wandb.save('checkpoint.pth')
      # You may find it interesting to exlplore Wandb Artifcats to version your models
# run.finish()


Epoch 1/20: 
Train Acc 50.0000%	 Train Loss 0.6970	 Learning Rate 0.0002


Val Acc 48.4375%	 Val Loss 0.6980
Adjusting learning rate of group 0 to 2.8911e-04.


Train:   6%|▌         | 38/622 [00:00<00:10, 55.04it/s, acc=48.5577%, loss=0.6995, lr=0.0003, num_correct=303]